In [ ]:
from fastai.text.all import *

In [ ]:
lang = 'de'
num_tweets = 500000
wiki = f'{lang}wiki'
base_path = Path('data')
path = base_path/wiki
data_path = path/'germeval'

# Preprocessing

Remove links and @-mentions from tweets

In [ ]:
def clean_text(text):
    text = re.sub('@\w+', ' ', text)
    text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    text = ' '.join(text.split())
    return text

## Training-Data GermEval 2018 (train & test) + 2019 (train)

GermEval 2018: https://github.com/uds-lsv/GermEval-2018-Data  
GermEval 2019: https://projects.fzai.h-da.de/iggsa/data-2019/

In [ ]:
names = ['text','label','label_fine']

df_train1 = pd.read_csv(f'{data_path}/germeval2018/germeval2018.test.txt',
                sep ='\t', names=names)
df_train2 = pd.read_csv(f'{data_path}/germeval2018/germeval2018.training.txt',
                sep ='\t', names=names)
df_train3 = pd.read_csv(f'{data_path}/germeval2019/germeval2019.training_subtask1_2_korrigiert.txt',
                sep = '\t', names=names)
df_train4 = pd.read_csv(f'{data_path}/germeval2019/germeval2019.training_subtask3.txt',
                sep = '\t', names=[*names,'label3'])

In [ ]:
df_train4.drop('label3', axis=1, inplace=True)

In [ ]:
df_train = pd.concat([df_train1,df_train2,df_train3], sort=False)

In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)

In [ ]:
df_train['is_valid'] = False

# Test-Data GermEval2019

In [ ]:
df_test = pd.read_csv(data_path/'germeval2019/germeval2019GoldLabelsSubtask1_2.txt',
                sep ='\t', names=names)

In [ ]:
df_test['text'] = df_test['text'].apply(clean_text)

In [ ]:
df_test['is_valid'] = True

In [ ]:
df_class = pd.concat([df_test,df_train], sort=False)

In [ ]:
df_class.to_csv(data_path/'data_classification.csv', index=False)

In [ ]:
df_class

,text,label,label_fine,is_valid
0,Komisch das die Realitätsverweigerung immer von linken erbärmlichen Correctiv Accounts ausgeht...,OFFENSE,INSULT,True
1,"CDU steht seid Strauss, Kohl, Schäuble, vdLeyen für Korruption, Schwarzgeld, und dies überall...",OFFENSE,ABUSE,True
2,"Die Gesichter, Namen, Religion der Täter würde zur Aufklärung beitragen...",OTHER,OTHER,True
3,"Wie verwirrt muss man sein um sich zu weigern die Tatsachen anzuerkennen, das dieser Hass von Ferkel importiert wurde....was bezweckt man damit sich selber so zu belügen, unverständlich",OFFENSE,ABUSE,True
4,"Weil die AfD den Ferkelfunk abschaffen will, deshalb haben die Linken Zecken, Schmarotzer Angst für ihr kacken arbeiten zu müssen, deshalb sind die Führer treu und senden jeden Dreck, Lüge...",OFFENSE,ABUSE,True
...,...,...,...,...
3975,"250 Menschen auf der Demonstration gegen das Dublin 2 Abkommen auf der Demo der selbstorganisierten ,,Refugees for Change'' #Refugeeswelcome",OTHER,OTHER,False
3976,Erneut Massaker an Kurdische ZivilistInnen durch islamistische ISIS. YXK ruft bundesweit zu Demonstrationen auf. #Rojava #ISIS #syrien,OTHER,OTHER,False
3977,Hunderte Refugees haben die Grenze zur spanischen Exklave Melilla überwunden. Freedom of Movement is everybodies right! #Rassismus #NoBorder,OTHER,OTHER,False
3978,"Heute ab 17:00 Uhr an der Alten Oper FFM: Kundgebung/Picknick/Musik zum Jahrestag der Gezi-Proteste #Gezi, #Taksim, #Solidarität, #Erdogan",OTHER,OTHER,False


# Additional Tweets for LM-finetuning

Source: https://zenodo.org/record/3633935#.YEVnVi24jzA

In [ ]:
df_tweet = pd.read_csv(data_path/'detweet'/'tweets.txt', sep=';', header=None)
df_tweet.columns = ['id','text']
df_tweet = df_tweet.drop('id', axis=1)

In [ ]:
df_tweet = df_tweet[:num_tweets]
df_tweet['text'] = df_tweet['text'].apply(clean_text)
len(df_tweet)

500000

In [ ]:
df_lm = pd.concat([df_train, df_tweet], sort=False)
df_lm.drop(['label','label_fine'], axis=1, inplace=True)
df_lm.drop('is_valid', axis=1, inplace=True)

In [ ]:
df_lm

,text
0,"Meine Mutter hat mir erzählt, dass mein Vater einen Wahlkreiskandidaten nicht gewählt hat, weil der gegen die Homo-Ehe ist ☺"
1,"Meine Reaktion; |LBR| Nicht jeder Moslem ist ein Terrorist. Aber jeder Moslem glaubt an Überlieferungen, die Gewalt und Terror begünstigen."
2,"#Merkel rollt dem Emir von #Katar, der islamistischen Terror unterstützt, den roten Teppich aus.Wir brauchen einen sofortigen #Waffenstopp!"
3,„Merle ist kein junges unschuldiges Mädchen“ Kch....... 😱 #tatort
4,Asylantenflut bringt eben nur negatives für Deutschland. Drum Asylanenstop und Rückführung der Mehrzahl.
...,...
499995,"🇪🇺 Keine Lust auf #Europa? Zurück zu nationalen Grenzen? Drei junge #Europäer wollten dabei nicht länger zusehen. Und gründeten #Volt. Eine Partei, die Europa einen soll. ⏩ #Europawahl2019"
499996,"Morgen um 11 Uhr geht die große Sprayer-Aktion an der Bahnunterführung in #Langen los. Wir sind total gespannt auf die Kunstwerke, die dort entstehen und verabschieden uns schon mal von den alten Schmierereien. Mehr Infos: #Graffiti #StreetArt"
499997,Neuigkeit: Gesundheitsgefahr: Krebshilfe fordert Werbeverbot für E-Zigaretten #news
499998,Hundeanhänger 925 Silber und recyceltes Holz sabina #Tchotchke #kreativerTchotchke #handgefertigt Cane ciondolo in argento 925 e sabina legno riciclato #gioielli #gioiellicreativi #fattoamano


In [ ]:
df_lm.to_csv(data_path/'data_lm.csv', index=False)